# EDA for QA4MRE dataset

## Imports and Setup

In [147]:
import csv
import requests
import xml.etree.ElementTree as ET
#import stanford_parser
import nltk
import pandas as pd
import re
#from stanford_parser import sentence_split

In [148]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

## Input .XML file, organize .CSV

In [149]:
# create element tree object 
def parse_file(xmlfiles,yrs,filename):
   
    # open a file for writing

    QA_data = open(filename, 'w')

    # create the csv writer object

    csvwriter = csv.writer(QA_data)

    # write column names
    headings = ['YR','T_ID','T_NAME','R_ID','D_ID','D_STR','Q_ID','Q_STR','A_ID','A_STR','CORRECT']
    csvwriter.writerow(headings)
    
    for i,xmlfile in enumerate(xmlfiles):
        print("Opening...",xmlfile)
        
        tree = ET.parse(xmlfile)
        yr = yrs[i]
  
        # get root element 
        root = tree.getroot() 

        #rows = []

        for topic in root.iter('topic'):
            t_id = topic.attrib['t_id']
            t_name = topic.attrib['t_name']
            #print(topic.attrib)

            for read_test in topic.iter('reading-test'):
                r_id = read_test.attrib['r_id']
                #print("\nreading test",r_id)
                doc = read_test.find('doc')
                d_id = doc.attrib['d_id']
                d_str = doc.text#.encode(#'utf-8')

                for questions in read_test.iter('q'):
                    q_id = questions.attrib['q_id']
                    question = questions.find('q_str').text#.encode('utf-8')
                    #print(question)

                    for answers in questions.iter('answer'):
                        rows = []

                        if ('correct' in answers.attrib):
                            correct = 1
                        else:
                            correct = 0
                        #print(correct)
                        a_id = answers.attrib['a_id']
                        a_text = answers.text#.encode('utf-8')
                        #print(a_text)

                        rows.append(yr)
                        rows.append(t_id)
                        rows.append(t_name)
                        rows.append(r_id)                
                        rows.append(d_id)
                        rows.append(d_str)
                        rows.append(q_id)
                        rows.append(question)
                        rows.append(a_id)
                        rows.append(a_text)
                        rows.append(correct)

                        csvwriter.writerow(rows)

        print("Closing...",xmlfile)

    print("Parsing complete!")
    QA_data.close()

In [150]:
#parse_file(test_file,[2011],'./data/qa4mre/qa4mre_tester.csv')

Opening... ./data/qa4mre/2011/QA4MRE-2011-EN_GS.xml
Closing... ./data/qa4mre/2011/QA4MRE-2011-EN_GS.xml
Parsing complete!


In [151]:
test_file = ['./data/qa4mre/2011/QA4MRE-2011-EN_GS.xml']
files = ['./data/qa4mre/2011/QA4MRE-2011-EN_GS.xml','./data/qa4mre/2012/QA4MRE-2012-EN_GS_SYNC.xml',
         './data/qa4mre/2012/QA4MRE-2012-EN_GS.xml','./data/qa4mre/2013/QA4MRE-2013-EN_GS.xml']
years = [2011,2012,2012,2013] # THIS NEEDS TO BE UPDATED AS FILES ARE ADDED/REMOVED

In [ ]:
## DON'T REDO THIS, JUST LOAD NOW
parse_file(files,years,'./data/qa4mre/qa4mre.csv')

In [ ]:
# FILES TO SAVE FOR LATER
'./data/qa4mre/2012/QA4MRE-2012_BIOMEDICAL_GS.xml','./data/qa4mre/2013/QA4MRE-2013_BIO_GS-RUN.xml',

## Load Data for EDA

In [154]:
qa4mre_df = pd.read_csv("./data/qa4mre/qa4mre.csv") #update as needed

In [155]:
qa4mre_df.head()

,YR,T_ID,T_NAME,R_ID,D_ID,D_STR,Q_ID,Q_STR,A_ID,A_STR,CORRECT
0,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,1,What event caused Annie Lennox to commit herse...,1,the imprisonment of Nelson Mandela at Robben I...,0
1,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,1,What event caused Annie Lennox to commit herse...,2,the closing ceremony of Nelson Mandela's Found...,0
2,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,1,What event caused Annie Lennox to commit herse...,3,the meeting with Youssou N'Dour,0
3,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,1,What event caused Annie Lennox to commit herse...,4,the racial segregation in South Africa,0
4,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,1,What event caused Annie Lennox to commit herse...,5,Nelson Mandela's conference to the world press,1


In [156]:
qa4mre_df.groupby(['T_ID']).size()

T_ID
1    960
2    970
3    990
4    700
dtype: int64

In [157]:
qa4mre_df.groupby(['YR','T_ID']).size()

YR    T_ID
2011  1       200
      2       200
      3       200
2012  1       400
      2       400
      3       400
      4       400
2013  1       360
      2       370
      3       390
      4       300
dtype: int64

In [180]:
qa4mre_df.groupby(['T_ID','T_NAME','YR']).size()

T_ID  T_NAME             YR  
1     AIDS               2011    200
                         2012    400
                         2013    360
2     Climate Change     2011    200
                         2012    400
                         2013    370
3     Music and Society  2011    200
                         2012    400
                         2013    390
4     Alzheimer          2012    400
                         2013    300
dtype: int64

In [185]:
qa4mre_df.groupby(['Q_ID']).size()

Q_ID
1     300
2     300
3     300
4     300
5     300
6     300
7     300
8     300
9     300
10    300
11     80
12     80
13     80
14     80
15     80
16     60
17     60
18     60
19     25
20     15
dtype: int64

In [36]:
qa4mre_df.groupby(['CORRECT','A_ID']).size()

CORRECT  ANS_ID
0        1         582
         2         586
         3         593
         4         583
         5         552
1        1         142
         2         138
         3         131
         4         141
         5         172
dtype: int64

In [160]:
qa4mre_df.groupby(['CORRECT','YR','A_ID']).size()

CORRECT  YR    A_ID
0        2011  1        89
               2        98
               3        97
               4        95
               5       101
         2012  1       258
               2       254
               3       256
               4       248
               5       264
         2013  1       235
               2       234
               3       240
               4       240
               5       187
1        2011  1        31
               2        22
               3        23
               4        25
               5        19
         2012  1        62
               2        66
               3        64
               4        72
               5        56
         2013  1        49
               2        50
               3        44
               4        44
               5        97
dtype: int64

In [178]:
qa4mre_df.groupby(['CORRECT','T_NAME','A_ID']).size()

T_NAME             A_ID
AIDS               1       192
                   2       192
                   3       192
                   4       192
                   5       192
Alzheimer          1       140
                   2       140
                   3       140
                   4       140
                   5       140
Climate Change     1       194
                   2       194
                   3       194
                   4       194
                   5       194
Music and Society  1       198
                   2       198
                   3       198
                   4       198
                   5       198
dtype: int64

## Process String Text

In [161]:
qa4mre_df.insert(6,'D_LEN',0)
qa4mre_df.insert(9,"Q_LEN",0)
qa4mre_df.insert(12,"A_LEN",0)

In [162]:
qa4mre_df.head()

,YR,T_ID,T_NAME,R_ID,D_ID,D_STR,D_LEN,Q_ID,Q_STR,Q_LEN,A_ID,A_STR,A_LEN,CORRECT
0,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,0,1,What event caused Annie Lennox to commit herse...,0,1,the imprisonment of Nelson Mandela at Robben I...,0,0
1,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,0,1,What event caused Annie Lennox to commit herse...,0,2,the closing ceremony of Nelson Mandela's Found...,0,0
2,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,0,1,What event caused Annie Lennox to commit herse...,0,3,the meeting with Youssou N'Dour,0,0
3,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,0,1,What event caused Annie Lennox to commit herse...,0,4,the racial segregation in South Africa,0,0
4,2011,1,AIDS,1,1,Annie Lennox Why I am an HIVAIDS activist I'm ...,0,1,What event caused Annie Lennox to commit herse...,0,5,Nelson Mandela's conference to the world press,0,1


In [163]:
for i,row in qa4mre_df.iterrows():
    qa4mre_df.at[i,'D_LEN'] = len(nltk.word_tokenize(row['D_STR']))
    qa4mre_df.at[i,'Q_LEN'] = len(nltk.word_tokenize(row['Q_STR']))
    qa4mre_df.at[i,'A_LEN'] = len(nltk.word_tokenize(row['A_STR']))

### Passage Length

In [164]:
qa4mre_df['D_LEN'].max()

4305

In [165]:
qa4mre_df['D_LEN'].min()

958

In [166]:
qa4mre_df['D_LEN'].mean() # This will become a problem for 2013, with variable # of questions

1991.3052486187846

### Question Length

In [167]:
qa4mre_df['Q_LEN'].max()

36

In [168]:
qa4mre_df['Q_LEN'].min()

4

In [169]:
qa4mre_df['Q_LEN'].mean() # Each question is represented 5 times, for each answer

12.113259668508288

### Answer Length

In [170]:
qa4mre_df['A_LEN'].max()

41

In [171]:
qa4mre_df['A_LEN'].min()

1

In [172]:
qa4mre_df['A_LEN'].mean() # Each question is represented 5 times, for each answer

4.702209944751381

In [177]:
qa4mre_df.groupby(['CORRECT']).max()

,YR,T_ID,T_NAME,R_ID,D_ID,D_STR,D_LEN,Q_ID,Q_STR,Q_LEN,A_ID,A_STR,A_LEN
CORRECT,,,,,,,,,,,,,
0,2013,4,Music and Society,16,16,"Elizabeth Pisani: Sex, drugs and HIV -- let's ...",4305,20,"Why, when playing at the house of Cardinal Ott...",36,5,you could work harder,22
1,2013,4,Music and Society,16,16,"Elizabeth Pisani: Sex, drugs and HIV -- let's ...",4305,20,"Why, when playing at the house of Cardinal Ott...",36,5,young people could listen to pop outside,41
